In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# auto reload the objects
%load_ext autoreload
%autoreload 2
import re
import pandas as pd
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
import numpy as np
import os
#os.getcwd()
# enable interactive plotly in notebook
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
!pip install umap-learn
import sklearn.datasets
from scipy.stats import stats
import requests
import pandas as pd
import umap as UMAP
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
outputdir = '/content/drive/MyDrive/o-miner/output/'

Mounted at /content/drive/


In [ ]:
clinical = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_clinical_sample.txt",sep="\t",skiprows=[i for i in range(1, 3)])
clinical.columns = clinical.iloc[1,:]
clinical = clinical.iloc[2:,:].reset_index(drop=True)
clinical.head()


1,PATIENT_ID,SAMPLE_ID,ONCOTREE_CODE,CANCER_TYPE,CANCER_TYPE_DETAILED,...,MSI_SCORE_MANTIS,MSI_SENSOR_SCORE,SOMATIC_STATUS,TMB_NONSYNONYMOUS,TISSUE_SOURCE_SITE
0,TCGA-02-2466,TCGA-02-2466-01,GBM,Glioblastoma,Glioblastoma Multiforme,...,0.2855,0.86,Matched,3.366666667,MD Anderson Cancer Center
1,TCGA-02-2470,TCGA-02-2470-01,GBM,Glioblastoma,Glioblastoma Multiforme,...,0.2735,0.02,Matched,1.7,MD Anderson Cancer Center
2,TCGA-02-2483,TCGA-02-2483-01,GBM,Glioblastoma,Glioblastoma Multiforme,...,0.2721,0.3,Matched,1.5,MD Anderson Cancer Center
3,TCGA-02-2485,TCGA-02-2485-01,GBM,Glioblastoma,Glioblastoma Multiforme,...,0.2728,0.15,Matched,1.833333333,MD Anderson Cancer Center
4,TCGA-02-2486,TCGA-02-2486-01,GBM,Glioblastoma,Glioblastoma Multiforme,...,0.2683,0.04,Matched,1.9,MD Anderson Cancer Center


In [ ]:
clinical_clean = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/TCGA_142_clinical_May6.txt",sep="\t")
clinical_clean.head()

,PATIENT_ID,SAMPLE_ID,Gene_Expression_Subtype,G-CIMP_Methylation,IDH1_Mutation,...,DSS,PFI.time,PFI,DFS.time,DFS
0,TCGA-02-2485,TCGA-02-2485-01,Classical,non-G-CIMP,WT,...,0.0,186.0,1,186.05,1
1,TCGA-02-2486,TCGA-02-2486-01,Mesenchymal,non-G-CIMP,WT,...,1.0,618.0,1,494.10,0
2,TCGA-06-1804,TCGA-06-1804-01,Classical,non-G-CIMP,WT,...,1.0,414.0,1,414.80,0
3,TCGA-06-2557,TCGA-06-2557-01,Mesenchymal,non-G-CIMP,WT,...,1.0,33.0,1,33.55,0
4,TCGA-06-2558,TCGA-06-2558-01,Proneural,non-G-CIMP,WT,...,NaN,380.0,0,381.25,0


# Second layer: transcriptome 

In [ ]:
transcriptome = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt",sep="\t")
transcriptome = transcriptome.loc[~transcriptome.Hugo_Symbol.isna()]
transcriptome=transcriptome.drop(['Entrez_Gene_Id'],axis = 1).T
transcriptome.columns = transcriptome.iloc[0,:]
transcriptome = transcriptome.iloc[1:,:]
transcriptome.head()
transcriptome.to_csv(outputdir+"transcriptome.txt",sep="\t",index= True)

Hugo_Symbol,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,...,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-02-0047-01,-0.3974,-0.0826,-2.0708,NaN,0.4351,...,-0.8546,0.4807,-0.9953,0.8076,-0.3677
TCGA-02-0055-01,1.0246,-1.8728,-0.7532,NaN,-2.1438,...,0.815,-0.7129,1.0233,-1.152,0.3233
TCGA-02-2483-01,0.8181,0.2359,-2.0708,NaN,-1.0198,...,3.3626,0.4862,-0.4944,-0.4085,0.2522
TCGA-02-2485-01,0.9404,1.6038,1.086,NaN,-0.1857,...,-1.4604,-0.0589,-0.4856,0.3293,0.2242
TCGA-02-2486-01,-0.9364,0.3519,-2.0708,NaN,-0.6798,...,-1.4604,-2.0233,0.306,-1.0706,-0.9038


(160, 20518)

# First layer: fusion, mutation, methalation

In [ ]:
CNA = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_log2_cna.txt",sep="\t")
CNA = CNA.loc[~CNA.Hugo_Symbol.isna()]
CNA=CNA.drop(['Entrez_Gene_Id'],axis = 1).T
CNA.columns = CNA.iloc[0,:]
CNA = CNA.iloc[1:,:]
CNA.head()
CNA.to_csv(outputdir+"CNA.txt",sep="\t",index= True)

In [ ]:
fusion = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_fusions.txt",sep="\t")
fusion=fusion.groupby(['Tumor_Sample_Barcode','Fusion']).count().reset_index()
fusion_df = pd.DataFrame(index=fusion.Tumor_Sample_Barcode.unique(),columns=fusion.Fusion.unique())
for idx in range(0,fusion.shape[0]):
  row = fusion.iloc[idx,:]
  fusion_df.loc[row['Tumor_Sample_Barcode'],row['Fusion']] = 1
fusion_df=fusion_df.fillna(0)
fusion_df.head()
fusion_df.to_csv(outputdir+"fusions.txt",sep="\t",index= True)

In [ ]:
methylation = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_methylation_hm27_hm450_merged.txt",sep="\t")
methylation = methylation.drop(['ENTITY_STABLE_ID','DESCRIPTION','TRANSCRIPT_ID'],axis = 1).T
methylation.columns = methylation.iloc[0,:]
methylation = methylation.iloc[1:,:]
methylation.head()
methylation.to_csv(outputdir+"methylation.txt",sep="\t",index= True)

# third layer: microbiome

In [ ]:
import pandas as pd
outputdir = '/content/drive/MyDrive/o-miner/output/'
microbiome = pd.read_csv("/content/drive/MyDrive/o-miner/dataset/gbm_tcga_pan_can_atlas_2018/data_microbiome.txt",sep="\t")
microbiome = microbiome.drop(['NAME','DESCRIPTION','URL'],axis = 1).T
microbiome.columns = microbiome.iloc[0,:]
microbiome = microbiome.iloc[1:,:]
microbiome.head()
microbiome.to_csv(outputdir+"microbiome.txt",sep="\t",index= True)